In [1]:
#!pip install --quiet langchain-anthropic langchain-neo4j cyVer langchain-google-genai json-repair "numpy<2"

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from tqdm import tqdm
import pandas as pd

from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI

from utils import (
    _value_sanitize,
    extract_json_from_markdown,
    sampling_query,
    validate_cypher,
    process_database,
    process_all_examples_with_limit,
    convert_datetime
)
from prompts import (
    system_prompt,
    simple_system_prompt,
)

# Generate dataset

In [4]:
# LLM selection
models = [ChatAnthropic(model='claude-opus-4-20250514'), ChatGoogleGenerativeAI(model="gemini-2.5-pro")]

# Database selection (for demo database)
db_url = "neo4j+s://demo.neo4jlabs.com"
databases = [
    "companies",
    "twitch", 
    "network",
    "northwind",
    "ClinicalKnowledgeGraph"
]


In [ ]:
simple_batch_count = 1 # Number of iterations for simple queries
multi_batch_count = 3 # Number of iterations complex queries

output = []

for model in models:
    print(model.model)
    for credential in tqdm(databases, desc="Processing databases"):
        # Simple question
        database_records = process_database(
            credential, db_url, model, simple_batch_count, simple_system_prompt
        )
        output.extend(database_records)

        database_records = process_database(
            credential, db_url, model, multi_batch_count, system_prompt
        )
        output.extend(database_records)

claude-opus-4-20250514


Iterations for companies: 100%|████████████████████| 1/1 [02:36<00:00, 156.98s/it]
                                                                                  
Iterations for twitch: 100%|███████████████████████| 1/1 [02:26<00:00, 146.25s/it]
                                                                                  
Iterations for network: 100%|██████████████████████| 1/1 [02:40<00:00, 160.22s/it]
                                                                                  
Iterations for northwind: 100%|████████████████████| 1/1 [02:15<00:00, 135.06s/it]
                                                                                  
Iterations for ClinicalKnowledgeGraph: 100%|███████| 1/1 [05:50<00:00, 350.89s/it]
                                                                                  
Processing databases: 100%|█████████████████████| 5/5 [2:02:17<00:00, 1467.40s/it]


models/gemini-2.5-pro


Iterations for companies: 100%|████████████████████| 1/1 [02:36<00:00, 156.71s/it]
                                                                                  
Iterations for companies:   0%|                             | 0/3 [00:00<?, ?it/s]

# Generate text answers

In [ ]:
qa_model = ChatAnthropic(model='claude-3-5-haiku-latest')

In [ ]:
# Generate text-based answers
await process_all_examples_with_limit(output, qa_model)

In [ ]:
df = pd.DataFrame.from_records(output)
df.head(5)

In [ ]:
with open("generated_dataset.json", "w") as f:
    json.dump(output, f, indent=2, default=convert_datetime)